In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 61.7 MB/s eta 0:00:00
     ━━━━━━

In [2]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, Trainer, pipeline
from peft import LoraConfig
from datasets import Dataset
from langchain.prompts.prompt import PromptTemplate

from trl import SFTTrainer
from peft import PeftModel

import pandas as pd

In [3]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
new_model = "./finetune_models/llama2"

In [4]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map={"":0}
)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [7]:
df_data = pd.read_csv("finetune_data_llama2b.csv")

In [8]:
df_data.head()

,instruction,question,sql_tbl_1,sql_tbl_2,answer
0,\nYou are an agent designed to interact with a...,Calculate the distance between the 'user' and ...,\n CREATE TABLE user (\n user_id INT...,\n CREATE TABLE user_reference (\n u...,"SELECT target_id, reference_id, distancetype, ..."
1,\nYou are an agent designed to interact with a...,Determine the distance between the 'item' and ...,\n CREATE TABLE item (\n item_id INT...,\n CREATE TABLE item_reference (\n i...,"SELECT target_id, reference_id, distancetype, ..."
2,\nYou are an agent designed to interact with a...,Find the distance between the 'inventory' and ...,\n CREATE TABLE inventory (\n produc...,\n CREATE TABLE inventory_reference (\n ...,"SELECT target_id, reference_id, distancetype, ..."
3,\nYou are an agent designed to interact with a...,Compute the distance between the 'user' and 'u...,\n CREATE TABLE user (\n user_id INT...,\n CREATE TABLE user_reference (\n u...,"SELECT target_id, reference_id, distancetype, ..."
4,\nYou are an agent designed to interact with a...,Calculate the distance between the 'transactio...,\n CREATE TABLE transaction (\n tran...,\n CREATE TABLE transaction_reference (\n ...,"SELECT target_id, reference_id, distancetype, ..."


In [10]:
prompt_template = PromptTemplate(
    input_variables=["instruction", "sql_table_1", "sql_table_2", "question", "answer"], template="<s>[INST] <<SYS>>{instruction}. Here is structure of table 1 {sql_table_1}, table 2 {sql_table_2}<</SYS>>{question}[/INST]{answer}</s>"
)

prompt_data = []
for i, row in df_data.iterrows():
  prompt_data.append(prompt_template.format(instruction=row.instruction, sql_table_1=row.sql_tbl_1, sql_table_2=row.sql_tbl_2, question=row.question, answer=row.answer))

dataset = Dataset.from_dict({"inputs": prompt_data})

In [11]:
print(dataset[2].get('inputs'))

<s>[INST] <<SYS>>
You are an agent designed to interact with a SQL database.
Given an input question, 
create a syntactically correct teradatasql query to run, then look at the results of the query and return the answer.
. Here is structure of table 1 
    CREATE TABLE inventory (
        product_id INT PRIMARY KEY,
        name VARCHAR(255),
        description TEXT,
        quantity INT,
        price DECIMAL(10, 2)
    );
    , table 2 
    CREATE TABLE inventory_reference (
        product_id INT PRIMARY KEY,
        name VARCHAR(255),
        description TEXT,
        quantity INT,
        price DECIMAL(10, 2)
    );
    <</SYS>>Find the distance between the 'inventory' and 'inventory_reference' tables using TD_VectorDistance.[/INST]SELECT target_id, reference_id, distancetype, cast(distance as decimal(36,8)) as distance FROM TD_VECTORDISTANCE (
    ON inventory AS TargetTable
    ON inventory_reference AS ReferenceTable DIMENSION
    USING
        TargetIDColumn('product_id')
   

In [12]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.config.use_cache = False
model.config.pretraining_tp = 1

In [17]:
peft_params = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [21]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True
)

In [22]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="inputs",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/76 [00:00<?, ? examples/s]

In [23]:
trainer.train()

Step,Training Loss
1,2.532800
2,2.618100
3,2.371800
4,2.354900
5,2.477900
6,2.033800
7,2.261900
8,1.980800
9,2.067500
10,1.817900


TrainOutput(global_step=152, training_loss=0.3708199826314261, metrics={'train_runtime': 226.0911, 'train_samples_per_second': 0.672, 'train_steps_per_second': 0.672, 'total_flos': 2440450877030400.0, 'train_loss': 0.3708199826314261, 'epoch': 2.0})

In [24]:
new_model = "llama2_taradata"

trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('llama2_taradata/tokenizer_config.json',
 'llama2_taradata/special_tokens_map.json',
 'llama2_taradata/tokenizer.model',
 'llama2_taradata/added_tokens.json',
 'llama2_taradata/tokenizer.json')

In [25]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
new_model = "llama2_taradata"

In [ ]:
# 1. Load based model
base_model = AutoModelForCausalLM.from_pretrained(model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

In [ ]:
# 2. Load new fine-tuned model. Then merge this two model
finetuned_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = finetuned_model.merge_and_unload()

In [ ]:
# 3. Load tokenizer,
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# 4. Define pipeline
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer, max_length=2000)

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["instruction", "sql_table_1", "sql_table_2", "question"], template="<s>[INST] <<SYS>>{instruction}. Here is structure of table 1 {sql_table_1}, table 2 {sql_table_2}<</SYS>>{question}[/INST]")


instruction = """You are an agent designed to interact with a SQL database.\nGiven an input question,
create a syntactically correct teradatasql query to run, then look at the results of the query and return the answer."""

question = "Calculate the separation between the 'item' and 'item_reference' tables using TD_VectorDistance."

sql_table_1 = """
    CREATE TABLE item (
        item_id INT PRIMARY KEY,
        name VARCHAR(255),
        description TEXT,
        price DECIMAL(10, 2),
        stock_quantity INT,
        category VARCHAR(100),
        weight INT,
        color_id INT,
        height INT,
        width INT
    );
"""

sql_table_2 = """
    CREATE TABLE item_reference (
        item_id INT PRIMARY KEY,
        name VARCHAR(255),
        description TEXT,
        price DECIMAL(10, 2),
        stock_quantity INT,
        category VARCHAR(100),
        weight INT,
        color_id INT,
        height INT,
        width INT
    );
"""

input = prompt_template.format(instruction=instruction, sql_table_1=sql_table_1, sql_table_2=sql_table_2, question=question)

In [ ]:
input

In [ ]:
# 5. Run the pipeline
result = pipe(input)

In [ ]:
print(result[0]['generated_text'].split("/INST")[1][1:])